In [ ]:
import model
import tensorflow as tf
import numpy as np
import pickle
import constants as const

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open("testing_data.pickle", "rb") as input_file:
    testing_data = pickle.load(input_file)

In [3]:
with open("vocabulary.pickle", "rb") as input_file:
    voc = pickle.load(input_file)

In [4]:
emb_mat = np.load("/mnt/word_embedding_matrix.npy")

edm = model.EncoderDecoderModel(emb_mat)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [5]:
def find_max_length(lst):
    length = max((len(e) for e in lst))
    return length

def convert_word_to_embedding_index(word, voc):
    if word in voc:
        return voc[word]
    else:
        return 0

In [11]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, "model/rnn_model")
    print("load sucessfully")
    
    batch_i = 0
    start_list = []
    end_list = []
    print(len(testing_data))
    while batch_i < len(testing_data):
        start = batch_i
        
        end = batch_i + const.BATCH_SIZE

        if end > len(testing_data):
            end = len(testing_data)
            
        q_list = []
        c_list = []

        for ins in testing_data[start: end]:
            q_list.append(list(map(lambda x: convert_word_to_embedding_index(x, voc), ins['question'])))
            c_list.append(list(map(lambda x: convert_word_to_embedding_index(x, voc), ins['context'])))
            
        while len(q_list) != const.BATCH_SIZE and len(c_list) != const.BATCH_SIZE:
            q_list.append([0])
            c_list.append([0])

        # padding to a matrix by '0'
        max_q = find_max_length(q_list)
        for i in q_list:
            i.extend([0] * (max_q - len(i)))
        batch_q = np.asarray(q_list)

        max_c = find_max_length(c_list)
        for i in c_list:
            i.extend([0] * (max_c - len(i)))
        batch_c = np.asarray(c_list)
        
        start_point = tf.argmax(edm.fc_1, axis=1)
        end_point = tf.argmax(edm.fc_2, axis=1)

        c_s, c_e = sess.run([start_point, end_point], feed_dict={edm.context_input: batch_c,
                                                          edm.question_input: batch_q,
                                                          edm.dropout_keep_prob: 1.0
                                                          })
        print(c_s, c_e)
        print(c_s.shape, c_e.shape)
        start_list.extend(c_s)
        end_list.extend(c_e)
        batch_i += const.BATCH_SIZE

INFO:tensorflow:Restoring parameters from model/rnn_model
load sucessfully
3618
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(128,) (128,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

KeyboardInterrupt: 

In [10]:
print(len(start_list))
print(len(end_list))

90306
90306
